# Single Cell Biology 
## Mini project : RNA-seq of fly ovaries dataset 

### Imports 

In [2]:
import scanpy as sc
import anndata as ad
import pandas as pd
import matplotlib.pyplot as plt

### Data loading 

In [3]:
data = pd.read_csv('ovary.tsv', sep="\t", index_col=0)
adata = ad.AnnData(data.T)
adata.var_names = data.index  # Gene names
adata.obs_names = data.columns  # Cell names
adata

AnnData object with n_obs × n_vars = 7549 × 11943